# Educational RAG

## Interface conversacional para recuperação de informações a partir de documentos

Este projeto usa RAG (Retrieval Augmented Generation) para garantir a acurácia das respostas.

## Setup inicial

In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver  
from langchain.tools import tool
from langchain.agents import create_agent

In [ ]:
conversational_model = "gpt-4o-mini"
sentence_transformers_model = "sentence-transformers/all-mpnet-base-v2"
db_name = "../data/educational_db"
knowledge_base = "../data/knowledge-base/*"
nearest_neighbors = 2

In [ ]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'key-if-not-using-dotenv')
os.environ['HF_API_KEY'] = os.getenv('HF_API_KEY', 'key-if-not-using-dotenv')

## Carga de documentos

In [ ]:
folders = glob.glob(knowledge_base)

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

print(f"Total de documentos carregados: {len(documents)}")


## Configuração de chunks

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total de chunks: {len(chunks)}")
print(f"Tipos de documentos encontrados: {set(doc.metadata['doc_type'] for doc in documents)}")

## Configuração de embeddings

In [ ]:
# embeddings = OpenAIEmbeddings()

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=sentence_transformers_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print(f"Configuração do embeddings criado:\n {embeddings}")

## Criação de banco vetorial

In [ ]:
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"Banco vetorial criado com {vectorstore._collection.count()} documentos (vetores)")
print(f"Os {count:,} vetores estão organizados em {dimensions:,} dimensões")

## Exemplo de recuperação de dados no banco vetorial

In [ ]:
query = "Please explain what Insurellm is in a couple of sentences"
retrieved_docs = vectorstore.similarity_search(query, nearest_neighbors)
print(retrieved_docs)

## Visualização do banco vetorial

In [ ]:
import importlib
import my_visualizer
importlib.reload(my_visualizer)
from my_visualizer import visualizer2d

visualizer2d(collection)

## Configuração do chat com LangChain

In [ ]:
model = init_chat_model(conversational_model, temperature=0.7)

memory = InMemorySaver()

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectorstore.similarity_search(query,k=nearest_neighbors)
    serialized = "\n\n".join(
        (f"metadata: {doc.metadata}\npage_content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

tools = [retrieve_context]

system_prompt = """
   You are an expert in answering only accurate questions about Insurellm, the Insurance Tech company.
   Give brief, accurate answers. If you don't know the answer, say so.
   Do not make anything up if you haven't been provided with relevant context.
 """ 

agent = create_agent(
    model=model,
    checkpointer=memory,
    tools=tools,
    system_prompt=system_prompt,
)

## Acionamento do agente

In [ ]:
config = {"configurable": {"thread_id": "1"}}
message = "Please explain what Insurellm is in a couple of sentences"
messages = {"role": "user", "content": message}
result = agent.invoke({"messages": [messages]}, config)
result["messages"][-1].content

## Criação de função para acionamento do chat

In [ ]:
def chat(message, history):
    messages = {"role": "user", "content": message}
    result = agent.invoke({"messages": [messages]}, config)
    return result["messages"][-1].content

## Configuração do Gradio com a interface do chat

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)